[SF-DST] Booking reviews

Описание:
Представьте, что вы работаете датасаентистом в компании Booking. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов нахождения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель играет нечестно, и его стоит проверить. Вам поставлена задача создать такую модель.

Описание:
Представьте, что вы работаете датасаентистом в компании Booking. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов нахождения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель играет нечестно, и его стоит проверить. Вам поставлена задача создать такую модель.

In [ ]:
Подготовка рабочего пространства

import pandas as pd
import numpy as np

DATA_DIR = '/content/'
df_train = pd.read_csv(DATA_DIR+'hotels_train.csv')               # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv')                 # датасет для предсказания
#sample_submission = pd.read_csv(DATA_DIR+'/submission.csv')      # самбмишн Сабмишн (англ. submission) — это победа в поединке по единоборствам,

In [1]:
import pandas as pd
import numpy as np
# DATA_DIR = '/content/'
df_train = pd.read_csv('hotels_train1.csv')               # датасет для обучения
df_test = pd.read_csv('hotels_test1.csv')                 # датасет для предсказания
#sample_submission = pd.read_csv(DATA_DIR+'/submission.csv')      # самбмишн

Столбец с названием reviewer_score обычно представляет собой числовой показатель оценки, выставленной рецензентом (например, оценка фильма, продукта, услуги и т.п.). Это может быть рейтинг или балл, отражающий мнение или качество с точки зрения обозревателя.
В зависимости от контекста данных reviewer_score может содержать:
Числовые значения (например, от 0 до 10 или 0 до 100),
Баллы или оценки, выставленные человеком или системой,
Количественную меру качества или удовлетворённости.
Часто такой столбец используется для анализа или фильтрации данных по значению оценки.
Если речь о конкретном DataFrame или наборе данных, reviewer_score — это колонка с числовыми оценками или рейтингами, которые можно использовать для аналитики, сортировки, вычисления средних значений и других статистических операций

In [2]:
# В pandas метод sample() используется для получения случайной выборки из строк DataFrame.
# Это означает, что с помощью sample() можно:
# Выбрать случайное количество строк (например, 5 случайных строк).
# Выбрать случайную часть DataFrame, указав долю (например, 10% строк).
# Перемешать строки всего DataFrame, если взять frac=1 (100% строк в случайном порядке).
# Пример использования:
# df.sample(n=5)       # 5 случайных строк из df
# df.sample(frac=0.1)  # 10% строк случайно из df
# df.sample(frac=1)    # перемешать все строки df
df_train['sample'] = 1           # помечаем где у нас трейн
df_test['sample'] = 0            # помечаем где у нас тест
df_test['reviewer_score'] = 0    # в тесте у нас нет значения reviewer_score, мы его должны предсказать, поэтому пока просто заполняем нулями ( reviewer_score обычно представляет собой числовой показатель оценки)
reviews_df = pd.concat([df_test, df_train], ignore_index=True, sort=False)
# reviews_df = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем train Для объединения df_test и df_train с помощью pd.concat() вместо устаревшего append()
# Объяснение изменений:
# pd.concat() принимает список DataFrame для объединения.
# Параметр ignore_index=True сбрасывает индексы в итоговом DataFrame.
# Параметр sort=False предотвращает автоматическую сортировку столбцов, оставляя порядок неизменным.
# Таким образом, код с concat() полностью заменит строку с append() и reset_index() за один вызов.

df = df.append(new_row, ignore_index=True) 
Ошибка 'DataFrame' object has no attribute 'append' возникает потому, что метод append() для объектов pandas DataFrame был устаревшим начиная с версии 1.4.0, а в версии Pandas 2.0 был полностью удалён. Этот метод раньше использовался для добавления строк к DataFrame, но теперь он больше не поддерживается.
df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
Таким образом, ошибка означает, что версия Pandas современная (2.0 и новее), где метода append() уже нет, и нужно заменить этот вызов на concat() для правильного добавления строк или объединения DataFrame.

In [3]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 18 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               515738 non-null  object 
 1   additional_number_of_scoring                515738 non-null  int64  
 2   review_date                                 515738 non-null  object 
 3   average_score                               515738 non-null  float64
 4   hotel_name                                  515738 non-null  object 
 5   reviewer_nationality                        515738 non-null  object 
 6   negative_review                             515738 non-null  object 
 7   review_total_negative_word_counts           515738 non-null  int64  
 8   total_number_of_reviews                     515738 non-null  int64  
 9   positive_review                             515738 non-null  object 
 

In [4]:
# объединяем положительные и отрицательные отзывы
reviews_df["review"] = reviews_df["negative_review"] + reviews_df["positive_review"]
# создаём метку для плохого отзыва (1 - True, 0 - False)
reviews_df["is_bad_review"] = reviews_df["reviewer_score"].apply(lambda x: 1 if x < 5 else 0)
# выбираем только релевантные столбцы
reviews_df = reviews_df[["review", "is_bad_review", "reviewer_score", "sample"]]
reviews_df.head()

,review,is_bad_review,reviewer_score,sample
0,Would have appreciated a shop in the hotel th...,1,0.0,0
1,No tissue paper box was present at the roomNo...,1,0.0,0
2,Pillows Nice welcoming and service,1,0.0,0
3,No Negative Everything including the nice upgr...,1,0.0,0
4,No Negative Lovely hotel v welcoming staff,1,0.0,0


Sample данные

In [5]:
reviews_df = reviews_df.sample(frac = 0.1, replace = False, random_state=42)

In [6]:
# удалим 'No Negative' или 'No Positive' из текста
reviews_df["review"] = reviews_df["review"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))

- import nltk — это команда в Python, которая импортирует библиотеку NLTK (Natural Language Toolkit).
NLTK — одна из самых популярных и мощных библиотек для обработки естественного языка (Natural Language Processing, NLP) в Python. Она предоставляет множество инструментов и ресурсов для работы с текстом: токенизацию, стемминг, лемматизацию, разметку частей речи, синтаксический разбор, корпуса и словари, а также алгоритмы для анализа текста и машинного обучения.
Эту библиотеку используют, когда надо анализировать, обрабатывать и понимать человеческий язык в программных приложениях, например, для создания чат-ботов, анализа тональности отзывов, автоматического перевода и других задач NLP.

- from nltk.corpus import wordnet означает импорт из библиотеки NLTK модуля WordNet.
  
- nltk.corpus — это модуль библиотеки NLTK, который предоставляет доступ к разнообразным текстовым корпусам и словарным ресурсам для обработки естественного языка (NLP) в Python.
  
- WordNet — это большая лексическая база данных английского языка, представляющая собой своего рода электронный словарь и тезаурус. В ней слова объединены в синонимические группы (синсеты), каждая из которых имеет определение, примеры употребления, а также связи с другими словами (синонимы, антонимы и другие отношения).
NLTK предоставляет удобный интерфейс для работы с WordNet: можно получать определения слов, списки синонимов, примеры использования, искать антонимы, строить семантические сети и проводить другие лингвистические анализы.

- Лемматизация — это процесс приведения слова к его базовой, начальной форме, называемой леммой. Например, слова "бежал", "бегу" и "бежать" после лемматизации превратятся в одну лемму — "бежать". Этот метод используется в обработке естественного языка (NLP) и поисковых системах для того, чтобы компьютер мог "понимать" разные грамматические формы одного слова как единое целое, что повышает точность анализа текста. 

In [7]:
# https://webdevblog.ru/podhody-lemmatizacii-s-primerami-v-python/
# использование wordnet для анализа частей речи

import nltk
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger')
# используется для загрузки в библиотеку NLTK предобученной модели для частеречной разметки текста — POS-теггера (Part-of-Speech tagger), 
# основанного на алгоритме усреднённого персептрона (averaged perceptron).

nltk.download('wordnet')
# означает загрузку в NLTK лексической базы данных WordNet.

nltk.download('stopwords')
# загружает в NLTK набор списков стоп-слов для разных языков.
# Стоп-слова — это часто встречающиеся в языке слова (например, предлоги, союзы, частицы: «и», «в», «не», «что»), 
# которые обычно не несут важной смысловой нагрузки и часто удаляются из текста при его обработке и анализе.
# NLTK не включает эти списки автоматически, поэтому их нужно скачать отдельно с помощью nltk.download('stopwords').

def get_wordnet_pos(pos_tag):
# Функция get_wordnet_pos(pos_tag) обычно используется в связке с библиотекой NLTK для корректной лемматизации слов с учётом их части речи (POS — Part Of Speech).
# Что делает эта функция:
# Принимает POS-тег в формате Penn Treebank (формат, который выдаёт функция nltk.pos_tag),
# Преобразует этот тег к формату POS-тегов, который ожидает лемматизатор WordNet из NLTK (wordnet.NOUN, wordnet.VERB, wordnet.ADJ, wordnet.ADV),
# Если тег не подходит под известные категории, возвращает wordnet.NOUN по умолчанию.
    
    if pos_tag.startswith('J'):
# означает проверку, начинается ли строка pos_tag с буквы 'J'.
# В контексте NLTK и POS-тегов (частей речи) это важно, потому что теги POS имеют формат Penn Treebank, где:
# Теги, начинающиеся с "J" обозначают прилагательные (adjectives).
# Например, "JJ" — прилагательное, "JJR" — прилагательное в сравнительной степени, "JJS" — прилагательное в превосходной степени.
# Поэтому эта проверка используется, чтобы определить, относится ли тег к прилагательным.
        
        return wordnet.ADJ
# это возвращение из функции значения wordnet.ADJ, означающего часть речи прилагательное (adjective) в формате, который понимает лемматизатор WordNet из библиотеки NLTK.
# В NLTK для лемматизации нужной является информация о части речи слова (POS). wordnet.ADJ — специальная константа, указывающая, что слово является прилагательным. 
# Это помогает лемматизатору правильно найти базовую форму слова с учётом его грамматической категории.
    
    elif pos_tag.startswith('V'):
# проверяет, начинается ли POS-тег (часть речи), заданный в формате Penn Treebank, с буквы "V".
# В системе тегов Penn Treebank все теги, начинающиеся на "V", обозначают глаголы.
# Примеры таких тегов:
# VB — глагол в базовой форме (например, "run"),
# VBD — глагол в прошедшем времени ("ran"),
# VBG — герундий или причастие настоящего времени ("running"),
# VBN — причастие прошедшего времени ("run"),
# VBP — глагол в настоящем времени (не третье лицо, "run"),
# VBZ — глагол в настоящем времени третьего лица единственного числа ("runs").
# Поэтому эта проверка используется для определения, что слово в предложении является глаголом, что позволяет, например, 
# корректно сопоставить тег с соответствующей частью речи для лемматизации или другой обработки языка.
        
        return wordnet.VERB
# это возвращение из функции значения wordnet.VERB, которое обозначает часть речи глагол (verb) в формате, который понимает лемматизатор WordNet из библиотеки NLTK.
# Это необходимо для лемматизации, при которой важно знать, к какой части речи относится слово, чтобы правильно привести его к базовой форме. wordnet.VERB указывает, 
# что слово является глаголом.

    elif pos_tag.startswith('N'):
# проверяет, начинается ли POS-тег, заданный в формате Penn Treebank, с буквы "N".
# В системе тегов Penn Treebank теги, начинающиеся с "N", обозначают существительные (nouns).
# Примеры таких тегов:
# NN — существительное в единственном числе (например, "cat"),
# NNS — существительное во множественном числе (например, "cats"),
# NNP — собственное имя (например, "John"),
# NNPS — собственные имена во множественном числе.
# Таким образом, эта проверка используется, чтобы определить, что часть речи слова является существительным, что важно, например, 
# для правильной обработки слова в лемматизации или синтаксическом разборе

        return wordnet.NOUN
# это возвращение из функции специального обозначения части речи существительное (noun) в формате, используемом WordNet из библиотеки NLTK.
# В контексте лемматизации WordNet требует указания части речи слова, чтобы правильно привести его к базовой форме. 
# wordnet.NOUN — это внутренний код, который обозначает, что слово является существительным.

    elif pos_tag.startswith('R'):
# проверяет, начинается ли POS-тег в формате Penn Treebank с буквы "R".
# В системе тегов Penn Treebank теги, начинающиеся с "R", обозначают наречия (adverbs).
# Примеры таких тегов:
# RB — наречие (например, "quickly"),
# RBR — сравнительная степень наречия (например, "faster"),
# RBS — превосходная степень наречия (например, "fastest").
# Таким образом, эта проверка нужна, чтобы определить, что часть речи слова — наречие, что важно для правильной лингвистической обработки, например, для лемматизации.

        return wordnet.ADV
# это возврат из функции значения wordnet.ADV, которое обозначает часть речи наречие (adverb) в формате, используемом лемматизатором WordNet библиотеки NLTK.
# В процессе лемматизации важно указывать часть речи слова для правильного нахождения базовой формы. wordnet.ADV указывает, что слово — это наречие.

    else:
        return wordnet.NOUN
# это возвращение из функции специальной константы wordnet.NOUN, которая обозначает часть речи существительное (noun) в формате, понятном лемматизатору WordNet из библиотеки NLTK.
# Это нужно для правильной лемматизации слов с учётом их категории. Указание, что слово — существительное, помогает алгоритму привести слово к базовой, словарной форме корректно.

import string
# импортирует встроенный в Python модуль string, который предоставляет удобные константы и функции для работы со строками.
# Модуль string содержит, например:
# Константы с предопределёнными наборами символов: ascii_letters (латинские буквы), digits (цифры), punctuation (знаки пунктуации) и др.
# Полезные функции для обработки строк, например, capwords() — для капитализации первой буквы каждого слова.
# Классы для форматирования и шаблонов, например Formatter и Template.
# Этот модуль часто используется для упрощения операций с текстом, например, проверки, какие символы есть в строке, форматирования сообщений, создания шаблонов для подстановки значений.
    
from nltk import pos_tag
# означает импорт функции pos_tag из библиотеки NLTK.
# Функция pos_tag используется для автоматической разметки текста, то есть для присвоения каждому слову в предложении его части речи (POS — Part Of Speech), 
# например, существительное, глагол, прилагательное и т.д.
# Процесс называется POS-тегингом (тегированием частей речи) и позволяет анализировать грамматическую структуру текста.

from nltk.corpus import stopwords
# означает импорт в программу набора списков стоп-слов из корпуса NLTK.
# Стоп-слова — это часто встречающиеся слова в языке (например, предлоги, союзы, частицы: «и», «в», «не», «что»), которые обычно не несут смысла и могут удаляться из текста при обработке для уменьшения шума.
# NLTK содержит готовые списки стоп-слов для разных языков, например, для русского, английского и других.
# Импортируя stopwords, можно получить эти списки и использовать для фильтрации текста

from nltk.tokenize import WhitespaceTokenizer
# означает импорт класса WhitespaceTokenizer из библиотеки NLTK.
# WhitespaceTokenizer — это простой токенизатор, который разбивает текст на токены (слова) по пробельным символам. Он разделяет текст на слова, используя пробелы, табуляции, 
# переводы строк и другие символы пробела как разделители.
# В отличие от более сложных токенизаторов, он не учитывает знаки препинания и другие символы, просто разбивает текст по пробелам. Это делает его быстрым, но менее точным для сложного анализа.

from nltk.stem import WordNetLemmatizer
# означает импорт класса WordNetLemmatizer из библиотеки NLTK.
# WordNetLemmatizer — это инструмент для лемматизации слов на английском языке. Лемматизация — это процесс приведения слова к его базовой или словарной форме (лемме) с учётом его части речи и контекста.
# В отличие от стемминга, который просто обрезает окончания слов, лемматизация учитывает грамматические и смысловые особенности слова, поэтому результат точнее.
# Например, слова "studies" и "studying" будут приведены к форме "study".

def clean_text(text):
# это объявление функции с именем clean_text, которая принимает один аргумент text. Обычно такая функция предназначена для очистки или предобработки текста.
# Функция очистки текста может выполнять следующие задачи:
# удаление лишних пробелов в начале и конце строки,
# удаление или замена специальных символов и знаков препинания,
# удаление стоп-слов,
# нормализация регистра (например, преобразование всего текста к нижнему регистру),
# удаление чисел, HTML-тегов, или других посторонних элементов,
# замена сокращений,
# и другие операции, упрощающие последующий анализ текста.
    
# приводим к нижнему регистру
    text = text.lower()
    
# разделение текста на слова и удаление знаков препинания
    text = [word.strip(string.punctuation) for word in text.split(" ")]
# означает следующее:
# text.split(" ") разбивает строку text на список слов, разделяя по пробелам.
# Для каждого слова из этого списка (for word in ...) выполняется операция word.strip(string.punctuation).
# Метод strip(string.punctuation) удаляет все знаки пунктуации (например, запятые, точки, восклицательные знаки и т.п.) с начала и конца слова.
# Результатом работы будет новый список слов без ведущих и завершающих знаков пунктуации.
# То есть, эта строка кода реализует очистку текста от пунктуации, разбивая текст на слова и очищая каждое слово от начальных и конечных знаков препинания.
    
# удаляем цифры из текста
    text = [word for word in text if not any(c.isdigit() for c in word)]
# делает следующее:
# Итерация по каждому слову в списке text.
# Для каждого слова проверяется условие: если в слове нет ни одной цифры (not any(c.isdigit() for c in word)).
# Вложенный генератор any(c.isdigit() for c in word) проверяет каждый символ слова word и возвращает True, если хотя бы один символ — цифра.
# Если условие истинно (то есть цифр в слове нет), слово добавляется в новый список.
# В итоге text становится списком слов без чисел.
# Это фильтрация списка слов, удаляющая все слова, которые содержат цифры.
    
# удаляем стоп-слова
    stop = stopwords.words('english')
# означает создание списка stop, который содержит предопределённый набор стоп-слов английского языка из библиотеки NLTK.
# Стоп-слова — это часто используемые слова (например, артикли, предлоги, союзы: "the", "is", "in", "and"), которые обычно не несут важной смысловой нагрузки 
# при анализе текста и часто удаляются для улучшения качества и эффективности обработки.
# Команда stopwords.words('english') загружает список таких слов на английском языке, предопределённый в NLTK.
# Этот список используется для фильтрации текста — исключения часто встречающихся, но малоинформативных слов из анализа, например, при классификации, 
# тематическом моделировании или анализе тональности.
    text = [x for x in text if x not in stop]
# означает создание нового списка из элементов списка text, при этом фильтруются все слова, которые содержатся в списке стоп-слов stop.
# То есть, из текста удаляются все слова, которые являются стоп-словами (например, предлоги, артикли, союзы), чтобы очистить текст от часто встречающихся и малоинформативных слов.
    
# удаляем пустые токены
    text = [t for t in text if len(t) > 0]
# означает создание нового списка, включающего только те элементы из списка text, у которых длина больше нуля.
# Проще говоря, это удаление из списка всех пустых строк или пустых элементов.
# Такой приём часто используется при предобработке текста для удаления пустых слов (пустых строк), которые могут появляться после разбиения текста или очистки.
    
# pos tag text
    pos_tags = pos_tag(text)
# означает вызов функции pos_tag из библиотеки NLTK для разметки списка слов text частями речи.
# Функция pos_tag принимает на вход последовательность токенов (слов) и возвращает список кортежей, где каждому слову сопоставлен его POS-тег (часть речи) согласно стандарту, например, Penn Treebank.
# Это позволяет понять грамматическую роль каждого слова в тексте — существительное, глагол, прилагательное, наречие и другие.
    
# лемматизация текста
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
# делает лемматизацию слов из списка pos_tags.
# Разбор компонентов:
# pos_tags — это список кортежей (слово, POS-тег) из функции pos_tag.
# t — само слово.
# t — POS-тег слова (например, 'NN', 'VB' и т.д.).
# get_wordnet_pos(t) — функция, которая преобразует POS-тег из формата Penn Treebank в формат, который понимает WordNet (например, 'N' для существительных, 'V' — для глаголов).
# WordNetLemmatizer().lemmatize(word, pos) — метод, который преобразует слово к базовой форме ("лемме") с учётом части речи.
# Итоговый список содержит лемматизированные формы всех слов.
# Таким образом, эта конструкция выполняет лемматизацию текста, учитывая грамматическую роль каждого слова, что делает нормализацию точнее и полезнее для последующего анализа

# удаляем слова из одной буквы
    text = [t for t in text if len(t) > 1]
# означает создание нового списка, который содержит только те элементы из списка text, длина которых больше 1.
# Другими словами, из списка удаляются все слова или элементы, состоящие из одного символа или пустые.
# Это часто используется в предобработке текста, чтобы отфильтровать слишком короткие слова, которые обычно малоинформативны.
    
# объединяем текст
    text = " ".join(text)
# означает объединение элементов списка text в одну строку, где элементы разделены пробелом.
# Метод join вызывается на строке-разделителе (в данном случае это пробел " "), а в качестве аргумента принимает список строк. Он создаёт новую строку, 
# в которой все элементы из списка соединены указанным разделителем.
    
    return(text)
# в Python используется внутри функции для возврата значения text обратно в вызывающий код.
# Оператор return завершает выполнение функции и передает указанное за ним значение (в данном случае text) туда, откуда была вызвана функция. 
# Это позволяет использовать результат вычислений функции в дальнейшем в программе.

# очистка текстовых данных

# reviews_df["review_clean"] = reviews_df["review"].apply(lambda x: clean_text(x))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Станислав\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Станислав\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Станислав\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# nltk.download()

In [ ]:
#Ошибка при выполнении строки
# reviews_df["review_clean"] = reviews_df["review"].apply(lambda x: clean_text(x))


In [10]:
# https://habr.com/ru/company/Voximplant/blog/446738/
# анализ тональности (настроения) текста отзыва

from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
reviews_df["sentiments"] = reviews_df["review"].apply(lambda x: sid.polarity_scores(x))
reviews_df = pd.concat([reviews_df.drop(['sentiments'], axis=1), reviews_df['sentiments'].apply(pd.Series)], axis=1)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Станислав\AppData\Roaming\nltk_data...
